In [1]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from spacy_sentiws import spaCySentiWS

nlp = spacy.load("de_core_news_sm")
nlp.add_pipe('sentiws', config={'sentiws_path': '/Users/changle/SentiWS_v2.0'})

In [ ]:
with open("Die Verwandlung.txt", "r", encoding="utf-8") as file:
    novel_text = file.read()

novel_doc = nlp(novel_text)

In [ ]:
keywords = ['Schwester', 'Schwesters', 'Grete']
data = {'Sentence': [], 'Average Sentiment Score': [], 'Tokens with Sentiment': [], 'Sentence Position': []}

for i, sentence in enumerate(novel_doc.sents):
    for keyword in keywords:
        if keyword.lower() in sentence.text.lower():
            sentiment_scores_sentence = [token._.sentiws for token in sentence if not token.is_punct and token._.sentiws is not None]
            average_sentiment = sum(sentiment_scores_sentence) / len(sentiment_scores_sentence) if sentiment_scores_sentence else None

            if average_sentiment is not None and not pd.isna(average_sentiment):
                tokens_with_sentiment = [(token.text, token._.sentiws) for token in sentence if not token.is_punct and token._.sentiws is not None]

                data['Sentence'].append(sentence.text)
                data['Average Sentiment Score'].append(average_sentiment)
                data['Tokens with Sentiment'].append(tokens_with_sentiment)
                data['Sentence Position'].append(i)

                plt.scatter(i, average_sentiment, color='blue')

In [ ]:
df = pd.DataFrame(data)
df.to_excel('Grete_metamorphosis.xlsx', index=False)

In [ ]:
df_fixed = pd.read_excel('fixed_Grete_metamorphosis.xlsx')

In [ ]:
def calculate_sentiment(sentence):
    doc = nlp(sentence)
    sentiment_scores = [token._.sentiws for token in doc if not token.is_punct and token._.sentiws is not None]
    tokens_with_sentiment = [(token.text, token._.sentiws) for token in doc if not token.is_punct and token._.sentiws is not None]
    total_score = sum(sentiment_scores)
    num_tokens_with_score = len(sentiment_scores)
    return sentiment_scores, tokens_with_sentiment, total_score, num_tokens_with_score

In [ ]:
df_fixed[['Sentiment Scores', 'Tokens With Sentiment', 'Total Score', 'Num Tokens With Score']] = df_fixed['Sentence'].apply(lambda x: pd.Series(calculate_sentiment(x)))

In [ ]:
df_fixed['Average Sentiment'] = df_fixed.apply(lambda row: row['Total Score'] / row['Num Tokens With Score'] if row['Num Tokens With Score'] != 0 else None, axis=1)

In [ ]:
df_fixed.to_excel('sentiws_before_revision.xlsx', index=False)